In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**2} MB")
else:
    print("GPU not available - check Runtime > Change runtime type > Hardware accelerator > GPU")

!pip install ultralytics -q

from ultralytics import YOLO
import os
from pathlib import Path
import yaml
from google.colab import files
import zipfile
import shutil

PyTorch version: 2.6.0+cu124
CUDA available: False
GPU not available - check Runtime > Change runtime type > Hardware accelerator > GPU
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s et

In [2]:
# Upload existing phone model
print("Upload your trained phone model (best.pt):")
uploaded_model = files.upload()

# Process uploaded model
for filename in uploaded_model.keys():
    if filename.endswith('.pt'):
        print(f"Model uploaded: {filename}")
        if filename != 'phone_model.pt':
            os.rename(filename, 'phone_model.pt')
        break

# Test the model
phone_model = YOLO('phone_model.pt')
print(f"Phone model loaded successfully")
print(f"Model classes: {phone_model.names}")

Upload your trained phone model (best.pt):


Saving best.pt to best.pt
Model uploaded: best.pt
Phone model loaded successfully
Model classes: {0: 'Mobile-phone'}


In [3]:
# Upload smartwatch dataset
print("Upload smartwatch dataset ZIP:")
uploaded_dataset = files.upload()

# Extract dataset
for filename in uploaded_dataset.keys():
    if filename.endswith('.zip'):
        print(f"Extracting: {filename}")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('smartwatch_raw')
        os.remove(filename)
        break

# Function to convert labels from class 0 to class 1
def convert_labels_to_class1(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    converted_count = 0

    for label_file in Path(input_dir).glob('*.txt'):
        with open(label_file, 'r') as f:
            content = f.read().strip()

        if content:
            lines = content.split('\n')
            converted_lines = []

            for line in lines:
                parts = line.split()
                if len(parts) >= 5:
                    # Change class from 0 to 1
                    parts[0] = '1'
                    converted_lines.append(' '.join(parts))

            # Write converted labels
            output_file = Path(output_dir) / label_file.name
            with open(output_file, 'w') as f:
                f.write('\n'.join(converted_lines))

            converted_count += 1
        else:
            # Empty label file
            output_file = Path(output_dir) / label_file.name
            with open(output_file, 'w') as f:
                f.write('')

    return converted_count

# Create processed dataset structure
dataset_dirs = [
    'smartwatch_dataset/train/images',
    'smartwatch_dataset/train/labels',
    'smartwatch_dataset/valid/images',
    'smartwatch_dataset/valid/labels',
    'smartwatch_dataset/test/images',
    'smartwatch_dataset/test/labels'
]

for dir_path in dataset_dirs:
    os.makedirs(dir_path, exist_ok=True)

# Copy images and convert labels
splits = ['train', 'valid', 'test']

for split in splits:
    raw_img_dir = f'smartwatch_raw/{split}/images'
    raw_lbl_dir = f'smartwatch_raw/{split}/labels'

    proc_img_dir = f'smartwatch_dataset/{split}/images'
    proc_lbl_dir = f'smartwatch_dataset/{split}/labels'

    # Copy images
    if os.path.exists(raw_img_dir):
        for img_file in Path(raw_img_dir).glob('*'):
            shutil.copy2(img_file, proc_img_dir)
        img_count = len(list(Path(proc_img_dir).glob('*')))
        print(f"{split} images copied: {img_count}")

    # Convert labels
    if os.path.exists(raw_lbl_dir):
        converted = convert_labels_to_class1(raw_lbl_dir, proc_lbl_dir)
        print(f"{split} labels converted (0->1): {converted}")

# Create new data.yaml for smartwatch with class 1
smartwatch_yaml = {
    'train': 'smartwatch_dataset/train/images',
    'val': 'smartwatch_dataset/valid/images',
    'test': 'smartwatch_dataset/test/images',
    'nc': 1,
    'names': ['smartwatch']
}

with open('smartwatch_data.yaml', 'w') as f:
    yaml.dump(smartwatch_yaml, f, default_flow_style=False)

print("Smartwatch dataset processed successfully")
print("Labels converted from class 0 to class 1")
print("smartwatch_data.yaml created")

# Cleanup
shutil.rmtree('smartwatch_raw')

Upload smartwatch dataset ZIP:


Saving Smart watch.v1i.yolov8.zip to Smart watch.v1i.yolov8.zip
Extracting: Smart watch.v1i.yolov8.zip
train images copied: 316
train labels converted (0->1): 316
valid images copied: 90
valid labels converted (0->1): 90
test images copied: 46
test labels converted (0->1): 46
Smartwatch dataset processed successfully
Labels converted from class 0 to class 1
smartwatch_data.yaml created


In [4]:
# Configure transfer learning from phone model to smartwatch
print("Starting transfer learning: phone model -> smartwatch detection")

# Training configuration
config = {
    'data': 'smartwatch_data.yaml',
    'epochs': 80,
    'batch': 16,
    'imgsz': 640,
    'device': 0,
    'project': 'smartwatch_training',
    'name': 'transfer_v1',
    'patience': 20,
    'save_period': 10,
    'workers': 2,
}

print(f"Training config: {config}")

# Load pre-trained phone model for transfer learning
model = YOLO('phone_model.pt')

# Start training
results = model.train(
    data=config['data'],
    epochs=config['epochs'],
    batch=config['batch'],
    imgsz=config['imgsz'],
    device=config['device'],
    project=config['project'],
    name=config['name'],
    patience=config['patience'],
    save_period=config['save_period'],
    workers=config['workers'],

    # Transfer learning settings
    freeze=6,  # Freeze first 6 layers

    # Reduced augmentation for transfer learning
    hsv_h=0.01,
    hsv_s=0.4,
    hsv_v=0.3,
    degrees=8.0,
    translate=0.05,
    scale=0.7,
    fliplr=0.5,

    # Lower learning rate for fine-tuning
    lr0=0.005,
    lrf=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=3,

    # Optimization
    optimizer='AdamW',
    cos_lr=True,
    cache=True,
)

print("Transfer learning completed")
print(f"Best model saved at: {config['project']}/{config['name']}/weights/best.pt")

Starting transfer learning: phone model -> smartwatch detection
Training config: {'data': 'smartwatch_data.yaml', 'epochs': 80, 'batch': 16, 'imgsz': 640, 'device': 0, 'project': 'smartwatch_training', 'name': 'transfer_v1', 'patience': 20, 'save_period': 10, 'workers': 2}
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
# Load best trained model
best_model_path = 'smartwatch_training/transfer_v1/weights/best.pt'
smartwatch_model = YOLO(best_model_path)

print("Validating smartwatch model...")

# Validate on test set
test_results = smartwatch_model.val(
    data='smartwatch_data.yaml',
    split='test',
    conf=0.25,
    iou=0.6,
    save_json=True
)

print("Validation Results:")
print(f"mAP50: {test_results.box.map50:.3f}")
print(f"mAP50-95: {test_results.box.map:.3f}")
print(f"Precision: {test_results.box.mp:.3f}")
print(f"Recall: {test_results.box.mr:.3f}")

# Test on sample images
test_images = list(Path('smartwatch_dataset/test/images').glob('*'))[:5]
print(f"\nTesting on {len(test_images)} sample images:")

for img_path in test_images:
    results = smartwatch_model(str(img_path), conf=0.3, save=True)

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            print(f"{img_path.name}: smartwatch detected, confidence: {conf:.3f}")
    else:
        print(f"{img_path.name}: no detection")

In [ ]:
# Create archive with results
shutil.make_archive('smartwatch_model_results', 'zip', 'smartwatch_training/transfer_v1')

print("Downloading results...")

# Download complete results
files.download('smartwatch_model_results.zip')

# Download just the model
files.download('smartwatch_training/transfer_v1/weights/best.pt')

print("Downloads completed")
print("Files:")
print("- smartwatch_model_results.zip (complete training results)")
print("- best.pt (trained smartwatch model)")